# Day 18: Put Options Deep Dive

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/astoreyai/money-talks/blob/main/class3_trading_strategies/week4_options_basics/day18_put_options.ipynb)

---

## Learning Objectives

1. **Master** put option mechanics and payoffs
2. **Understand** protective puts for hedging
3. **Learn** cash-secured puts for income/entry
4. **Calculate** put option risk/reward scenarios
5. **Apply** put strategies to real situations

---

## Lecture (30 minutes)

### Put Option Review

```
PUT OPTION FUNDAMENTALS
=======================

PUT = Right to SELL stock at strike price

BUYER (Long Put):
- Pays premium
- Wants stock to go DOWN
- Limited risk (premium)
- Large profit potential (to $0)

SELLER (Short Put):
- Receives premium
- Wants stock to stay flat or go UP
- Limited profit (premium)
- Large risk (stock to $0)
```

### Buying Puts (Long Puts)

```
LONG PUT STRATEGY
=================

OUTLOOK: Bearish (expect stock to fall)

SETUP:
Buy 1 Put at Strike K for Premium P

KEY NUMBERS:
Max Loss = Premium paid (P)
Breakeven = Strike - Premium (K - P)
Max Profit = Strike - Premium (if stock goes to $0)

EXAMPLE:
Stock at $175, Buy $170 Put for $3.00

Max Loss = $3.00 × 100 = $300
Breakeven = $170 - $3 = $167
Max Profit = ($167 - $0) × 100 = $16,700

AT EXPIRATION:
Stock at $160: Profit = ($167 - $160) × 100 = $700
Stock at $170: Loss = $300 (expires worthless)
Stock at $180: Loss = $300 (expires worthless)
```

### Protective Puts (Portfolio Insurance)

```
PROTECTIVE PUT STRATEGY
=======================

SETUP:
Own 100 shares of stock
+ Buy 1 Put option (insurance)

OUTLOOK: Bullish long-term, worried short-term

PURPOSE: Protect against downside while keeping upside

EXAMPLE:
Own 100 shares at $175
Buy $165 Put for $2.50

SCENARIOS AT EXPIRATION:

1. Stock at $185:
   - Stock gain: +$1,000
   - Put expires worthless: -$250
   - Net: +$750

2. Stock at $175 (unchanged):
   - Stock gain: $0
   - Put expires worthless: -$250
   - Net: -$250 (cost of insurance)

3. Stock at $150 (crash):
   - Stock loss: -$2,500
   - Put profit: ($165-$150-$2.50)×100 = +$1,250
   - Net: -$1,250 (protected from -$2,500!)

MAX LOSS = (Stock Cost - Strike) + Premium
= ($175 - $165) + $2.50 = $12.50/share = $1,250 max

Without put: Could lose $17,500 if stock goes to $0
With put: Max loss capped at $1,250
```

### Cash-Secured Puts

```
CASH-SECURED PUT STRATEGY
=========================

SETUP:
Sell 1 Put option
+ Hold cash to buy shares if assigned

"CASH-SECURED" = You have money to buy shares

OUTLOOK: Bullish/Neutral, want to buy cheaper

EXAMPLE:
Stock at $175
Sell $165 Put for $2.00
Hold $16,500 cash (165 × 100)

SCENARIOS AT EXPIRATION:

1. Stock at $180 (above strike):
   - Put expires worthless
   - Keep $200 premium
   - No shares bought
   - Try again next month!

2. Stock at $165 (at strike):
   - Could be assigned
   - Buy shares at $165
   - Effective cost: $165 - $2 = $163

3. Stock at $150 (below strike):
   - Assigned, buy at $165
   - Immediate loss: $165 - $150 = $15/share
   - But: Premium reduces to $13/share
   - Effective cost: $163

WIN-WIN STRATEGY:
- Stock stays up: Keep premium (income)
- Stock falls: Buy stock at discount
```

### Cash-Secured Put Mechanics

```
CASH-SECURED PUT TRADE MANAGEMENT
=================================

OPENING THE TRADE:
1. Identify stock you WANT to own
2. Choose strike at your target price
3. Sell put, collect premium
4. Hold cash for potential assignment

STRIKE SELECTION:
- Support levels make good strikes
- 5-10% below current price typical
- Lower strike = less premium but cheaper buy

EXPIRATION SELECTION:
- 30-45 days optimal
- Enough time value
- Not too long commitment

MANAGEMENT:
If stock rises:
- Put expires worthless, keep premium
- Sell another put

If stock falls to strike:
- Get assigned, buy shares
- Start selling covered calls!

If stock crashes:
- Buy shares at strike
- Loss = (Strike - Market) - Premium
- But you wanted to own anyway!

CAPITAL REQUIRED:
Cash = Strike Price × 100
$165 strike = $16,500 cash needed
```

### Put Selling Risks

```
CASH-SECURED vs NAKED PUTS
==========================

CASH-SECURED PUT (Acceptable risk):
- Have cash to buy shares
- If assigned, you own stock
- Max loss = Strike - Premium (stock to $0)
- Requires margin or cash

NAKED PUT (Higher risk):
- Don't have full cash
- Using margin leverage
- Margin calls possible
- Same theoretical max loss
- But capital efficiency different

KEY RULE:
Only sell puts on stocks you would
HAPPILY own at the strike price!

WORST CASE EXAMPLE:
Sell $50 Put for $2
Stock goes to $10
Loss = ($50 - $10 - $2) × 100 = $3,800
But: You now own stock worth $1,000
```

---

## Hands-On Practice (15 minutes)

In [ ]:
!pip install yfinance pandas numpy matplotlib -q
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
print("Libraries loaded!")

In [ ]:
def long_put_calculator(stock_price, strike, premium, contracts=1):
    """Calculate long put trade metrics."""
    shares = contracts * 100
    total_cost = premium * shares
    breakeven = strike - premium
    max_profit = (breakeven) * shares  # If stock goes to $0
    
    print(f"\n{'='*50}")
    print("LONG PUT CALCULATOR")
    print(f"{'='*50}")
    print(f"\nCurrent Stock Price: ${stock_price:.2f}")
    print(f"Strike Price: ${strike:.2f}")
    print(f"Premium: ${premium:.2f} per share")
    print(f"Contracts: {contracts} ({shares} shares)")
    
    print(f"\n{'-'*50}")
    print("TRADE METRICS:")
    print(f"{'-'*50}")
    print(f"Total Cost: ${total_cost:,.2f}")
    print(f"Breakeven: ${breakeven:.2f}")
    print(f"Max Loss: ${total_cost:,.2f} (100% of investment)")
    print(f"Max Profit: ${max_profit:,.2f} (if stock goes to $0)")
    
    # Required move
    move_to_be = (breakeven - stock_price) / stock_price * 100
    print(f"\nRequired Move to Breakeven: {move_to_be:.1f}%")
    
    return breakeven, total_cost

# Example long put
be, cost = long_put_calculator(stock_price=175, strike=170, premium=3.50, contracts=2)

In [ ]:
def plot_long_put_payoff(strike, premium):
    """Plot long put payoff diagram."""
    stock_range = np.linspace(strike * 0.5, strike * 1.3, 100)
    
    # Calculate payoff
    payoff = np.maximum(strike - stock_range, 0) - premium
    breakeven = strike - premium
    
    fig, ax = plt.subplots(figsize=(12, 7))
    
    # Plot payoff line
    ax.plot(stock_range, payoff, 'r-', lw=2.5, label='Long Put Payoff')
    
    # Fill areas
    ax.fill_between(stock_range, payoff, 0, where=payoff > 0, alpha=0.3, color='green', label='Profit Zone')
    ax.fill_between(stock_range, payoff, 0, where=payoff < 0, alpha=0.3, color='red', label='Loss Zone')
    
    # Reference lines
    ax.axhline(y=0, color='black', linestyle='-', lw=1)
    ax.axhline(y=-premium, color='red', linestyle='--', alpha=0.5)
    ax.axvline(x=strike, color='gray', linestyle='--', alpha=0.5, label=f'Strike: ${strike}')
    ax.axvline(x=breakeven, color='green', linestyle='--', alpha=0.5, label=f'Breakeven: ${breakeven}')
    
    # Max profit annotation
    max_profit = strike - premium
    ax.annotate(f'Max Profit: ${max_profit:.2f}\n(if stock → $0)', 
                xy=(strike * 0.55, max_profit * 0.7), fontsize=10, color='green')
    ax.annotate(f'Max Loss: -${premium}', xy=(strike * 1.15, -premium), fontsize=10, color='red')
    
    ax.set_xlabel('Stock Price at Expiration', fontsize=12)
    ax.set_ylabel('Profit/Loss per Share', fontsize=12)
    ax.set_title(f'Long Put Payoff Diagram\nStrike: ${strike} | Premium: ${premium}', fontsize=14)
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_long_put_payoff(strike=170, premium=3.50)

In [ ]:
def protective_put_calculator(shares_cost_basis, current_price, strike, premium, shares=100):
    """Calculate protective put (portfolio insurance) metrics."""
    total_premium = premium * shares
    total_investment = shares_cost_basis * shares
    
    print(f"\n{'='*60}")
    print("PROTECTIVE PUT (PORTFOLIO INSURANCE) CALCULATOR")
    print(f"{'='*60}")
    print(f"\nShares Owned: {shares} (Cost Basis: ${shares_cost_basis:.2f})")
    print(f"Current Stock Price: ${current_price:.2f}")
    print(f"Put Strike: ${strike:.2f}")
    print(f"Put Premium: ${premium:.2f} per share")
    
    # Calculations
    max_loss = (shares_cost_basis - strike + premium) * shares
    insurance_cost_pct = (total_premium / total_investment) * 100
    protection_level = strike / shares_cost_basis * 100
    
    print(f"\n{'-'*60}")
    print("PROTECTION METRICS:")
    print(f"{'-'*60}")
    print(f"Insurance Cost: ${total_premium:,.2f} ({insurance_cost_pct:.2f}% of position)")
    print(f"Protected Below: ${strike:.2f} ({protection_level:.1f}% of cost basis)")
    print(f"Max Loss (with put): ${max_loss:,.2f}")
    print(f"Max Loss (without put): ${total_investment:,.2f} (stock to $0)")
    print(f"Savings in Crash: ${total_investment - max_loss:,.2f}")
    
    # Scenarios
    print(f"\n{'-'*60}")
    print("SCENARIOS AT EXPIRATION:")
    print(f"{'-'*60}")
    
    scenarios = [current_price * 1.1, current_price, strike, current_price * 0.7, current_price * 0.5]
    
    for price in scenarios:
        stock_pnl = (price - shares_cost_basis) * shares
        put_payoff = max(0, strike - price) * shares
        total_pnl = stock_pnl + put_payoff - total_premium
        
        # Without protection
        unprotected_pnl = stock_pnl
        
        print(f"\nStock at ${price:.2f}:")
        print(f"  With Put: ${total_pnl:+,.2f}")
        print(f"  Without: ${unprotected_pnl:+,.2f}")
        print(f"  Difference: ${total_pnl - unprotected_pnl:+,.2f}")
    
    return max_loss, total_premium

# Example protective put
protective_put_calculator(
    shares_cost_basis=175, 
    current_price=175, 
    strike=165, 
    premium=3.00
)

In [ ]:
def cash_secured_put_calculator(current_price, strike, premium, shares=100):
    """Calculate cash-secured put metrics."""
    total_premium = premium * shares
    cash_required = strike * shares
    effective_buy_price = strike - premium
    
    print(f"\n{'='*60}")
    print("CASH-SECURED PUT CALCULATOR")
    print(f"{'='*60}")
    print(f"\nCurrent Stock Price: ${current_price:.2f}")
    print(f"Put Strike: ${strike:.2f}")
    print(f"Premium Received: ${premium:.2f} per share")
    
    print(f"\n{'-'*60}")
    print("TRADE SETUP:")
    print(f"{'-'*60}")
    print(f"Cash Required: ${cash_required:,.2f}")
    print(f"Premium Collected: ${total_premium:,.2f}")
    print(f"Return on Cash (if expires): {(total_premium/cash_required)*100:.2f}%")
    
    print(f"\n{'-'*60}")
    print("IF ASSIGNED:")
    print(f"{'-'*60}")
    print(f"Buy Price: ${strike:.2f}")
    print(f"Effective Cost Basis: ${effective_buy_price:.2f}")
    print(f"Discount from Current: {((current_price - effective_buy_price)/current_price)*100:.1f}%")
    
    # Scenarios
    print(f"\n{'-'*60}")
    print("SCENARIOS AT EXPIRATION:")
    print(f"{'-'*60}")
    
    scenarios = [current_price * 1.1, current_price, strike, strike * 0.9, strike * 0.8]
    
    for price in scenarios:
        if price > strike:
            # Put expires worthless
            result = "NOT ASSIGNED"
            pnl = total_premium
            shares_owned = False
        else:
            # Assigned
            result = "ASSIGNED"
            pnl = (price - effective_buy_price) * shares
            shares_owned = True
        
        print(f"\nStock at ${price:.2f}: {result}")
        if shares_owned:
            print(f"  Now own {shares} shares at ${effective_buy_price:.2f}")
            print(f"  Paper P/L: ${pnl:+,.2f}")
        else:
            print(f"  Keep premium: ${pnl:+,.2f}")
            print(f"  Return on cash: {(pnl/cash_required)*100:.2f}%")
    
    return effective_buy_price, total_premium

# Example cash-secured put
cash_secured_put_calculator(
    current_price=175, 
    strike=165, 
    premium=2.50
)

In [ ]:
def find_cash_secured_put_opportunities(ticker):
    """Find attractive cash-secured put opportunities."""
    stock = yf.Ticker(ticker)
    info = stock.info
    current_price = info.get('currentPrice', info.get('regularMarketPrice', 0))
    
    expirations = stock.options
    
    if len(expirations) == 0:
        print(f"No options available for {ticker}")
        return
    
    print(f"\n{'='*70}")
    print(f"CASH-SECURED PUT OPPORTUNITIES: {ticker}")
    print(f"{'='*70}")
    print(f"\nCurrent Price: ${current_price:.2f}")
    print(f"Note: Look for puts 5-15% below current price\n")
    
    # Analyze first 3 expirations
    for exp in expirations[:3]:
        try:
            options = stock.option_chain(exp)
            puts = options.puts
            
            # Filter OTM puts (5-15% below current price)
            otm_puts = puts[
                (puts['strike'] < current_price * 0.97) & 
                (puts['strike'] > current_price * 0.85) &
                (puts['bid'] > 0)
            ].copy()
            
            if len(otm_puts) == 0:
                continue
            
            print(f"{'-'*70}")
            print(f"Expiration: {exp}")
            print(f"{'-'*70}")
            
            for _, row in otm_puts.head(4).iterrows():
                strike = row['strike']
                premium = row['bid']  # Use bid for realistic fill
                
                # Calculate metrics
                discount = (current_price - strike) / current_price * 100
                premium_yield = premium / strike * 100
                effective_buy = strike - premium
                total_discount = (current_price - effective_buy) / current_price * 100
                
                print(f"\n  Strike: ${strike:.2f} ({discount:.1f}% below current)")
                print(f"  Premium (bid): ${premium:.2f} ({premium_yield:.2f}% yield)")
                print(f"  Effective buy price: ${effective_buy:.2f}")
                print(f"  Total discount if assigned: {total_discount:.1f}%")
        except Exception as e:
            pass

find_cash_secured_put_opportunities('AAPL')

In [ ]:
def plot_protective_put_payoff(cost_basis, strike, premium):
    """Plot protective put vs stock only comparison."""
    stock_range = np.linspace(cost_basis * 0.5, cost_basis * 1.3, 100)
    
    # Stock only P/L
    stock_pnl = stock_range - cost_basis
    
    # Protective put P/L
    put_value = np.maximum(strike - stock_range, 0)
    protected_pnl = stock_pnl + put_value - premium
    
    fig, ax = plt.subplots(figsize=(12, 7))
    
    # Plot both
    ax.plot(stock_range, stock_pnl, 'b--', lw=2, label='Stock Only', alpha=0.7)
    ax.plot(stock_range, protected_pnl, 'g-', lw=2.5, label='Stock + Protective Put')
    
    # Reference lines
    ax.axhline(y=0, color='black', linestyle='-', lw=1)
    ax.axvline(x=cost_basis, color='gray', linestyle=':', alpha=0.5)
    ax.axvline(x=strike, color='red', linestyle='--', alpha=0.5, label=f'Put Strike: ${strike}')
    
    # Max loss with protection
    max_loss = cost_basis - strike + premium
    ax.axhline(y=-max_loss, color='green', linestyle='--', alpha=0.5)
    
    # Annotations
    ax.annotate(f'Max Loss Protected: -${max_loss:.2f}', 
                xy=(strike * 0.6, -max_loss + 2), fontsize=10, color='green')
    ax.annotate(f'Insurance Cost: ${premium}', 
                xy=(cost_basis * 1.15, -premium - 2), fontsize=10)
    
    ax.set_xlabel('Stock Price at Expiration', fontsize=12)
    ax.set_ylabel('Profit/Loss per Share', fontsize=12)
    ax.set_title(f'Protective Put vs Unprotected Stock\nCost: ${cost_basis} | Put Strike: ${strike} | Premium: ${premium}', fontsize=14)
    ax.legend(loc='upper left')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_protective_put_payoff(cost_basis=175, strike=165, premium=3.00)

---

## Quiz

In [ ]:
quiz = [
    {"q": "A put option gives the buyer the right to?", "a": "B", 
     "opts": ["A) Buy stock at strike", "B) Sell stock at strike", 
              "C) Buy stock at market", "D) Sell stock at market"]},
    {"q": "Long put breakeven = ?", "a": "B",
     "opts": ["A) Strike + Premium", "B) Strike - Premium", 
              "C) Stock price - Premium", "D) Stock price + Premium"]},
    {"q": "A protective put combines?", "a": "C",
     "opts": ["A) Short stock + long call", "B) Long stock + short put", 
              "C) Long stock + long put", "D) Short stock + short put"]},
    {"q": "Cash-secured puts require?", "a": "A",
     "opts": ["A) Cash to buy shares if assigned", "B) Shares to sell", 
              "C) No capital", "D) A matching call option"]},
    {"q": "Best time to sell cash-secured puts?", "a": "D",
     "opts": ["A) When expecting stock to crash", "B) After buying shares", 
              "C) When IV is very low", "D) When willing to own stock at strike"]}
]

def run_quiz():
    for i, q in enumerate(quiz):
        print(f"\nQ{i+1}: {q['q']}")
        for opt in q['opts']:
            print(f"   {opt}")
    print("\n" + "="*50)
    print("ANSWERS: 1-B, 2-B, 3-C, 4-A, 5-D")
    
run_quiz()

---

## Summary

- **Long puts**: Bearish, limited loss, profit if stock falls
- **Breakeven** = Strike - Premium
- **Protective puts**: Insurance for stock you own
- **Cash-secured puts**: Sell puts to buy stock at discount
- **Only sell puts** on stocks you want to own

**Tomorrow**: Day 19 - Basic Options Strategies